#gensim word2vec

In [2]:
!pwd
%cd /content/drive/My Drive/Colab Notebooks/NLP

/content
/content/drive/My Drive/Colab Notebooks/NLP


In [3]:
%pip install mecab-python3
%pip install unidic-lite

     |████████████████████████████████| 488 kB 5.4 MB/s 
     |████████████████████████████████| 47.4 MB 47 kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658836 sha256=a4cfb928d86f38cd823e674af2cca69a3c187b0e1f7706b4bb3f6017918b211d
  Stored in directory: /root/.cache/pip/wheels/de/69/b1/112140b599f2b13f609d485a99e357ba68df194d2079c5b1a2
Successfully built unidic-lite


In [4]:
#　Reference https://www.pytry3g.com/entry/gensim-word2veReference 
import MeCab
import codecs
import urllib.parse as parser
import urllib.request as request
from bs4 import BeautifulSoup

tagger = MeCab.Tagger("-Owakati")
link = "https://ja.wikipedia.org/wiki/"
keyword = [
           "ロボット","ロケット","コンピュータ","人工知能","宇宙","機械","道具",
           "人間","目","口","耳","指","手","足","頭","首","声","腹","背","胃","腸",
           "動物","植物","猫","犬","鳩","魚","鳥","昆虫","猿","烏","ミミズ","ナメクジ","蟻","熊","クジラ","イルカ",
           "日本","東京","横浜","アジア","アメリカ","アフリカ","中近東","オセアニア","関東","関西",
           "電気","信号","電波","重力","原子","分子","量子",
           "紙","プラスチック","鉄","アルミ","ビニール","土","火","水",
           "聖徳太子","坂本龍馬","徳川家康","源義経","夏目漱石","宮沢賢治","斎藤茂吉",
           "サッカー","野球","ラグビー","プロレス","テニス","バレーボール","ホッケー","スキー","スケート",
           "台所","玄関","居間","窓","庭","カーテン","机","椅子","布団","毛布","寝室","屋根","壁","カーテン","風呂",
           "空","雲","雨","風","雪","台風","地震","津波","雪崩","津波",
           "飛行機","自動車","自転車","電車","機関車","バス","トラック",
           ] 

corpus = []
for word in keyword:
    #print(word)
    with request.urlopen(link + parser.quote_plus(word)) as response:
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, "lxml")
        p_tags = soup.find_all('p')
        for p in p_tags:
            corpus.append(tagger.parse(p.text).strip())

import re
corpus = [re.sub('[（）、。，「」［］『』0123456789]+','',sentence) for sentence in corpus]
print(len(corpus))
print(corpus[0:5])
with codecs.open("pwiki.txt", "w", "utf-8") as f:
    f.write("\n".join(corpus))


6143
['ロボット  チェコ 語 : robot  は  人 の 代わり に 何 等 か の 作業 を 自律 的 に 行う 装置  もしくは 機械 の こと ', '主に 以下 に 大別 する こと が 可能 で ある ', '近年 で は 無人 機  ドローン  を 半ば 自律 化 さ せ た もの [  ] も 存在 し  自動 運転 車 の 実現 が 視野 に 入っ て き て おり  SF の 世界 が 現実 の もの と なり つつ ある ', '生命 体 に 通常 以上 の 力 を 発揮 さ せる 方策 と し て 何 ら か の 人工 物 を 埋め込ん だり 置き換える など の 方策 を 採っ た 者 は 一般 に  サイボーグ  など と 呼ば れ 区別 さ れる こと が 多い ', 'この 言葉 が 初めて 用い られ た の は   年 に チェコスロバキア  当時  の 小説 家 カレル ・ チャペック が 発表 し た 戯曲  R . U . R .  ロッサム 万能 ロボット 商会   に おい て で ある が  この 作品 で は 現在 認知 さ れ て いる 金属 製 の 機械 で は なく  人間 と は 異なる 組成 の 肉体 と 人間 そっくり の 外見 を 持つ もの を  化学 的 合成 で 原形 質 を 使っ て 製作 し た もの で  現在 の SF で 言う バイオ ノ イド で ある ']


In [5]:
#　Reference https://www.pytry3g.com/entry/gensim-word2veReference 
import codecs
from gensim.models import word2vec

with codecs.open("pwiki.txt", "r", "utf-8") as f:
    corpus = f.read().splitlines()
corpus = [sentence.split() for sentence in corpus]
model = word2vec.Word2Vec(
    corpus, 
    size=128, #embedding dim
    min_count=10, # cut off count - use only words whose count is >=min_count
    window=8, # window size
    sg=1 # use skip-gram, not cbow
    )
model.save("pwiki.model")
model = word2vec.Word2Vec.load("pwiki.model")

print('vocab size: {}'.format(len(model.wv.vocab)))

vocab size: 4537


In [6]:
import torch
import torch.nn as nn

def comp(w1,w2):
  cos = nn.CosineSimilarity(dim=0)
  s = cos(torch.tensor(model.wv[w1]), torch.tensor(model.wv[w2]))
  return s.detach().item()

print('cos similarity b/w 人, ロボット -> {}'.format(comp('人','ロボット')))
print('cos similarity b/w 人, 機械 -> {}'.format(comp('人', '機械')))
print('cos similarity b/w 人, コンピュータ -> {}'.format(comp('人', 'コンピュータ')))
print('cos similarity b/w ロボット, 機械 -> {}'.format(comp('ロボット', '機械')))
print('cos similarity b/w 機械, コンピュータ -> {}'.format(comp('機械', 'コンピュータ')))

print('人に近い単語', end=': ')
print(*[" ".join([v, str("{:.2f}".format(s))]) for v, s in model.wv.most_similar(positive=["人"], topn=5)])
print('ロボットに近い単語', end=': ')
print(*[" ".join([v, str("{:.2f}".format(s))]) for v, s in model.wv.most_similar(positive=["ロボット"], topn=5)])
print('機械に近い単語', end=': ')
print(*[" ".join([v, str("{:.2f}".format(s))]) for v, s in model.wv.most_similar(positive=["機械"], topn=5)])
print('コンピュータに近い単語', end=': ')
print(*[" ".join([v, str("{:.2f}".format(s))]) for v, s in model.wv.most_similar(positive=["コンピュータ"], topn=5)])


cos similarity b/w 人, ロボット -> 0.19331489503383636
cos similarity b/w 人, 機械 -> 0.09271620213985443
cos similarity b/w 人, コンピュータ -> 0.24795791506767273
cos similarity b/w ロボット, 機械 -> 0.6857212781906128
cos similarity b/w 機械, コンピュータ -> 0.7661816477775574
人に近い単語: 日系 0.72 外国 0.72 民 0.70 在日 0.69 ブラジル 0.69
ロボットに近い単語: サービス 0.84 掃除 0.81 モーター 0.81 音楽 0.81 システム 0.80
機械に近い単語: 製 0.84 器具 0.84 用途 0.84 家電 0.82 加工 0.82
コンピュータに近い単語: システム 0.87 学習 0.86 計算 0.85 デジタル 0.85 手法 0.84
